A simple programme that apply momentum investing strategy,
Free sandbox data is used and is supplyed by IEX cloud API .

The programme screen through stocks in S&P500 and filter out top 20 stocks with greatest 1 year price return. Then number of stocks to buy is calculated for that list after inputing portfolio size

In [44]:
import numpy as np
import pandas as pd
import requests 
import math
from scipy import stats

In [45]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [46]:
#checking the api call
symbol = "AAPL"
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#data

In [47]:
# Define a function for making sublists
def chunks(lst,stepsize):
    for i in range(0, len(lst), stepsize):
        yield lst[i:i+stepsize]
        

In [48]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range (0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
column_names = ['Ticker','Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [49]:
# Batch api requests
final_df = pd.DataFrame(columns = column_names)

for symbol_string in symbol_strings :
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append( pd.Series([symbol,
                                               data[symbol]['quote']['latestPrice'],
                                               data[symbol]['stats']['year1ChangePercent'],
                                               'N/A'],
                                               index = column_names),
                                               ignore_index = True) 

        
        
#final_df

In [50]:
# Remove low-momentum stocks

final_df.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_df = final_df[:20]
final_df.reset_index(drop = True, inplace = True)
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,39.77,3.86842,N/A
1,FANG,109.92,3.19331,N/A
2,MRO,16.40,3.18424,N/A
3,OXY,34.50,2.83654,N/A
4,LB,83.40,2.3889,N/A
5,EOG,94.19,1.85749,N/A
6,COTY,8.35,1.7977,N/A
7,MCHP,73.86,1.76367,N/A
8,IT,328.51,1.6912,N/A
9,FTNT,324.63,1.67247,N/A


In [52]:
# A basic method to calculate no. of shares to buy

def portfolio():
    global portfolio_size
    portfolio_size = input('Size of the portfolio = ')
portfolio()    

Size of the portfolio = 1000000


In [53]:
size = float(portfolio_size)/len(final_df.index)
for i in range (0,len(final_df['Ticker'])):
    final_df.loc[i,'Number of Shares to Buy'] = math.floor(size/final_df['Price'][i])
    
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,39.77,3.86842,1257
1,FANG,109.92,3.19331,454
2,MRO,16.40,3.18424,3048
3,OXY,34.50,2.83654,1449
4,LB,83.40,2.3889,599
5,EOG,94.19,1.85749,530
6,COTY,8.35,1.7977,5988
7,MCHP,73.86,1.76367,676
8,IT,328.51,1.6912,152
9,FTNT,324.63,1.67247,154
